In [34]:
from dse.cluster import Cluster, GraphExecutionProfile, EXEC_PROFILE_GRAPH_DEFAULT
from dse.graph import GraphOptions, GraphProtocol
from dse_graph import DseGraph

from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P

from math import radians, cos, sin, asin, sqrt


In [2]:
graph_name = "trees"
ep = GraphExecutionProfile(graph_options=GraphOptions(graph_name=graph_name,
                                                      graph_protocol=GraphProtocol.GRAPHSON_3_0))
cluster = Cluster(execution_profiles={EXEC_PROFILE_GRAPH_DEFAULT: ep})
session = cluster.connect()
g = DseGraph.traversal_source(session=session)

In [59]:
def create_weighted_send_edges(sensor, g, label, key):

    # get the top 10 nearest sensors
    print(nearby_neighbors)
    # add an Weighted_Send edge with initialized weight of 0.10
    
    
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r


def top_neighbors(sensors, compare_list, num_neighbors):
    nearest_neighbors = {}
    for sensor in sensors:
        sensor_name = sensor.get("id")
        nearest_neighbors[sensor_name] = {}
        for entity in compare_list:
            if entity.get("id") != sensor_name:
                dist = haversine(sensor.get("long"),
                                          sensor.get("lat"),
                                          entity.get("long"),
                                          entity.get("lat"))
                nearest_neighbors[sensor_name][dist] = entity
        nearest_neighbors[sensor_name] = list(sorted(nearest_neighbors[sensor_name].items()))
        nearest_neighbors[sensor_name] = nearest_neighbors[sensor_name][0:num_neighbors]
    return nearest_neighbors

        
def init_eligibility_weights(g):
    ## Get all Sensors
    sensors = g.V().hasLabel("Sensor"). \
                    project("id", "lat", "long"). \
                      by("sensorName"). \
                      by("latitude"). \
                      by("longitude"). \
                      toList()
    towers = g.V().hasLabel("Tower"). \
                   project("id", "lat", "long"). \
                      by("towerName"). \
                      by("latitude"). \
                      by("longitude"). \
                      toList()
    sensor_sensor_neighbors = top_neighbors(sensors, sensors, 10)
    sensor_tower_neighbors = top_neighbors(sensors, towers, 10)

    return(sensor_sensor_neighbors, sensor_tower_neighbors )
    # for each sensor
    #for sensor in sensors:
        # initialize the sensor -- Weighted_Send --> sensor edges
        #create_weighted_send_edges(g, sensor, "Sensor", )
        # initialize the sensor -- Weighted_Send --> tower edges
        #create_weighted_send_edges(g, sensor, "Tower")


In [60]:
sensors = g.V().hasLabel("Sensor"). \
                    project("id", "lat", "long"). \
                      by("sensorName"). \
                      by("latitude"). \
                      by("longitude"). \
                      toList()
nearest_neigh  = top_neighbors(sensors, sensors, 10)


In [69]:
type(nearest_neigh["103318117"][0][1])

dict

In [33]:
create_weighted_send_edges(sensors[0], g, "Sensor", "sensorName")

NameError: name 'Geo' is not defined

In [8]:
test = {}
test.get("nada")

In [9]:
test.get("nada")

In [10]:
test

{}

In [14]:
val = test.get("a")
bal2 = test.get("B")

In [21]:
if (val and bal2) is not None:
    print("yes!")

In [35]:
test = {1:2, 2:3, 0:4}

In [36]:
test

{1: 2, 2: 3, 0: 4}

In [37]:
test = dict(sorted(test.items()))

In [38]:
test

{0: 4, 1: 2, 2: 3}